In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import pandas as pd

from key_drivers import (
    decompose_funnel_metrics,
    find_key_drivers,
    reduce_cat_columns,
)

pd.set_option("display.max_rows", 100)

# Define helper functions and variables

In [4]:
# Average price of an item purchased in each department
avg_prices = {
    "AUTOMOTIVE": 28,
    "BABY CARE": 9,
    "BEAUTY": 12,
    "BEVERAGES": 5,
    "BOOKS": 15,
    "BREAD/BAKERY": 6,
    "CELEBRATION": 10,
    "CLEANING": 5,
    "DAIRY": 3,
    "DELI": 8,
    "EGGS": 3,
    "FROZEN FOODS": 5,
    "GROCERY I": 4,
    "GROCERY II": 6,
    "HARDWARE": 20,
    "HOME AND KITCHEN I": 9,
    "HOME AND KITCHEN II": 16,
    "HOME APPLIANCES": 50,
    "HOME CARE": 5,
    "LADIESWEAR": 20,
    "LAWN AND GARDEN": 18,
    "LINGERIE": 25,
    "LIQUOR,WINE,BEER": 25,
    "MAGAZINES": 5,
    "MEATS": 13,
    "PERSONAL CARE": 4,
    "PET SUPPLIES": 10,
    "PLAYERS AND ELECTRONICS": 47,
    "POULTRY": 8,
    "PREPARED FOODS": 8,
    "PRODUCE": 1.5,
    "SCHOOL AND OFFICE SUPPLIES": 11,
    "SEAFOOD": 17,
}


# Simulate some more dynamic data
def promotions_to_profit_discount_factor(promotions, avg_price):
    discount_factor = np.log2(promotions + 1) / 100
    rarity = np.log2(avg_price + 1)
    return discount_factor * rarity


# Add some quick additional context - apologies if the pandas-fu is a bit wonky.
# Long and short of it is we want how many months since dataset begin we start
# seeing data
def categorise_openings(months_since_open: int) -> str:
    # I know this won't age well on a dynamic dataset, sue me
    if months_since_open == 0:
        return "already open"
    if months_since_open < 24:
        return "first_24"
    if months_since_open < 36:
        return "rush_open"
    return "brand_new"

# Construct the dataset

We'll import these steps in for subsequent notebooks.


In [5]:
# Read in the data and do type conversions
df_sales = pd.read_csv("train.csv")
df_transactions = pd.read_csv("transactions.csv")
df_stores = pd.read_csv("stores.csv")
df_stores.columns = [
    item if item.startswith("store") else "store_" + item for item in df_stores.columns
]

for item in [df_sales, df_transactions]:
    item["date"] = pd.to_datetime(item.date)

BASIC_COLUMNS = ["sales", "gross_profit", "net_profit"]

# Run some basic numbers to arrive at a profit figure - you'd normally pull this
# from a database or accounting system
df_sales["price"] = df_sales.family.map(avg_prices)
df_sales["gross_profit"] = df_sales.sales * df_sales.price

df_sales["profit_discount_factor"] = promotions_to_profit_discount_factor(
    df_sales.onpromotion, df_sales.price
)
df_sales["promotion_cost"] = df_sales.gross_profit * df_sales.profit_discount_factor
df_sales["net_profit"] = df_sales.gross_profit - df_sales.promotion_cost

# There's some weird stuff e.g. groceries spiking in and out, but eh its enough for
# a quick example. We also want to sum out the grocery categories, as we lack
# the data to use groceries as a sensible category (no join to transactions)
sales_totals = df_sales.groupby(["store_nbr", "date"])[BASIC_COLUMNS].sum()

df_sales = (
    sales_totals.join(
        df_transactions.set_index(["store_nbr", "date"]), on=["store_nbr", "date"]
    )
    .fillna(0)
    .reset_index()
)

df_sales["report_date"] = df_sales.date + pd.tseries.offsets.MonthEnd(0)
df_sales = (
    df_sales.groupby(["store_nbr", "report_date"])[BASIC_COLUMNS + ["transactions"]]
    .sum()
    .reset_index()
)

# Add some additional context to the stores to make the decomposition more
# interesting

store_openings = (
    (
        (
            df_sales[df_sales.net_profit != 0]
            .groupby("store_nbr")
            .report_date.first()
            .dt.to_period("M")
            - df_sales.report_date.min().to_period("M")
        ).apply(lambda x: x.n)
    )
    .map(categorise_openings)
    .rename("opening_time_cat")
)

df_stores["opening_time_cat"] = df_stores.store_nbr.map(store_openings.to_dict())

FUNNEL_VARIABLES = [
    "transactions",
    "items_per_transaction",
    "income_per_item",
    "profit_less_promotions_factor",
]

GROUP_COLS = [item for item in df_stores.columns if item != "store_nbr"]

# Convert to funnel analytics
df_sales["items_per_transaction"] = df_sales.sales / df_sales.transactions
df_sales["income_per_item"] = df_sales.gross_profit / df_sales.sales
df_sales["profit_less_promotions_factor"] = df_sales.net_profit / df_sales.gross_profit

# Truncate data slightly to get a "good read"
df_sales: pd.DataFrame = df_sales.loc[df_sales.report_date.lt("2017-01-01")]

# Apply the decomposition to ID key drivers

In [6]:
df_metrics = decompose_funnel_metrics(
    df_sales, "report_date", ["store_nbr"], FUNNEL_VARIABLES
)

# Remove all rows where the store is closed
df_metrics = df_metrics[~(df_metrics[FUNNEL_VARIABLES] == 0).all(axis=1)]

df_joined = (
    df_metrics.set_index("store_nbr")
    .join(df_stores.set_index("store_nbr"), on="store_nbr")
    .reset_index()
)


df_joined = reduce_cat_columns(df_joined, "store_nbr", GROUP_COLS, FUNNEL_VARIABLES)

key_drivers = find_key_drivers(
    df_joined,
    20,
    "store_nbr",
    GROUP_COLS,
    FUNNEL_VARIABLES,
)
key_drivers

[DrivingFactor(categories={'store_nbr': '45'}, total=0.055554055489387, vcount=47, id_count=1),
 DrivingFactor(categories={'store_nbr': '49'}, total=0.06180409601675288, vcount=47, id_count=1),
 DrivingFactor(categories={'store_city': 'Quito', 'store_state': 'Pichincha', 'store_type': 'A', 'store_cluster': '14', 'opening_time_cat': 'already open'}, total=0.11772982614095974, vcount=141, id_count=3),
 DrivingFactor(categories={'store_city': 'Quito', 'store_state': 'Pichincha', 'store_type': 'D', 'store_cluster': '8', 'opening_time_cat': 'already open'}, total=0.092708126035449, vcount=141, id_count=3),
 DrivingFactor(categories={'store_city': 'Quito', 'store_state': 'Pichincha', 'store_type': 'D', 'opening_time_cat': 'already open'}, total=0.06183388306964198, vcount=188, id_count=4),
 DrivingFactor(categories={'store_city': 'Guayaquil', 'store_state': 'Guayas', 'store_cluster': '10'}, total=0.05123551380080866, vcount=115, id_count=3),
 DrivingFactor(categories={'store_city': '(Machala